## EVALUATION DU MODELE XGBOOST POUR PLUMENET
### Thierry

In [1]:
# Insatllation des packages
! pip install scikit-learn
! pip install xgboost
! pip install matplotlib


[notice] A new release of pip is available: 23.0.1 -> 24.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.1
[notice] To update, run: pip install --upgrade pip


In [86]:
# Import des librairies
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split, RandomizedSearchCV
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [87]:
# Initialisation des variables d'environnement
plumnet_random_state=0

In [88]:
!pwd

python(39449) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


/Users/pauljonquet/Documents/Artefact/plumeNet/src/models/CNN


In [89]:
# Chargement des données
dataset = 'french_bird_db_50_100'
dir_path = os.path.join('.', dataset)
print(dir_path)

./french_bird_db_50_100


In [91]:
# # Image de test
# espece = 'anthus_trivialis'
# image_test = 'anthus_trivialis_0.jpg'
# file_path = os.path.join(dir_path, espece, image_test)
# print(file_path)

# img = plt.imread(file_path)
# plt.figure(figsize=(10,10))
# plt.axis('off')
# plt.imshow(img)
# plt.show()

In [92]:
# Format des images
# print(img.shape)

(270000,)


In [93]:
# ----------------------------------------------------- CHARGEMENT DU DATASET -------------------------------------------------------------

# Liste des répertoires = liste des espèces
repertoires = [nom for nom in os.listdir(dir_path) if os.path.isdir(os.path.join(dir_path, nom))]
repertoires = sorted(repertoires)

liste_images = []
liste_especes = []
X_constr = []
y_constr = []

espece_id = 0

max_images = 50


# Récupération du nom des images
for repertoire in repertoires :
    nb_images = 0
    liste_especes.append({'nom' : repertoire, 'id' : espece_id})
    dir_espece = os.path.join(dir_path, repertoire)
    
    for fichier in os.listdir(dir_espece) :
        if nb_images < max_images :
            #print(fichier)
            nom_fichier = os.path.join(dir_espece, fichier)
            img = plt.imread(nom_fichier)
            img = img.reshape(-1)
            #print(img.shape[0])
            if (img.shape[0] == 300*300*3) :
                X_constr.append(img)
                y_constr.append(espece_id)
                nb_images += 1
    espece_id += 1
liste_especes = pd.DataFrame(liste_especes)

In [94]:
print(nb_images," images")

50  images


In [95]:
# Construction de X, y
y = pd.DataFrame(y_constr)
print(y.shape)
X = np.vstack(X_constr)
X = pd.DataFrame(X)
print(X.shape)

(2500, 1)
(2500, 270000)


In [96]:
# Préparation des données - normalisation
print('Max =', X.values.max())

for ligne in range(len(X)) :
    X[ligne] = np.float16(X[ligne]/255)

Max = 255


In [97]:
y_shaped = np.array(y).reshape(y.shape[0])
y_shaped.shape

(2500,)

In [98]:
# Split
X_train, X_test, y_train, y_test = train_test_split(X, y_shaped, test_size=0.2, shuffle=True, random_state=plumnet_random_state)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(2000, 270000) (2000,) (500, 270000) (500,)


In [106]:
# Réduction de dimensions

pca = PCA(n_components=100)
pca.fit(X_train)
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

In [107]:
# Constrction du pipeline
pipeline = Pipeline([
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'))
])

In [108]:
# Définir la distribution des hyperparamètres pour la recherche aléatoire
# https://xgboost.readthedocs.io/en/stable/parameter.html

#param_dist = {
#    'xgb__n_estimators': [50, 100, 200],  # Nombre d'arbres de décision à entraîner
#    'xgb__max_depth': [3, 6, 8], # Profondeur maximale des arbres de décision
#    'xgb__learning_rate': [0.01, 0.1, 0.3, 0.4], # Taux d'apprentissage
#    'xgb__gamma': [0, 0.1, 0.2], # # Seuil de réduction de la perte pour une nouvelle partition
#    'xgb__reg_lambda': [1, 1.5, 2] # Taux de régularisation L2
#}

In [109]:
param_dist = {
    'xgb__n_estimators': [5, 10],  # Nombre d'arbres de décision à entraîner
    'xgb__max_depth': [3], # Profondeur maximale des arbres de décision
    'xgb__learning_rate': [0.1], # Taux d'apprentissage
    'xgb__gamma': [0.1], # # Seuil de réduction de la perte pour une nouvelle partition
    'xgb__reg_lambda': [1.5] # Taux de régularisation L2
}

In [110]:
X_train.shape, X_train_pca.shape, y_train.shape

((2000, 270000), (2000, 100), (2000,))

In [111]:
# Effectuer la recherche aléatoire avec validation croisée
nb_iterations = 5 # Nombre de combinaisons différentes d'hyperparamètres
nb_cv = 5 # Nombre de cross-validations
random_search = RandomizedSearchCV(pipeline, param_distributions=param_dist, n_iter=nb_iterations, cv=nb_cv, scoring='accuracy', random_state=plumnet_random_state, n_jobs=-1, verbose=2)
random_search.fit(X_train_pca, y_train)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


/Users/pauljonquet/.pyenv/versions/3.10.12/envs/Artefact/lib/python3.10/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 2 is smaller than n_iter=5. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
python(40812) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(40814) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(40816) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


[CV] END xgb__gamma=0.1, xgb__learning_rate=0.1, xgb__max_depth=3, xgb__n_estimators=5, xgb__reg_lambda=1.5; total time=   3.4s
[CV] END xgb__gamma=0.1, xgb__learning_rate=0.1, xgb__max_depth=3, xgb__n_estimators=5, xgb__reg_lambda=1.5; total time=   3.4s
[CV] END xgb__gamma=0.1, xgb__learning_rate=0.1, xgb__max_depth=3, xgb__n_estimators=5, xgb__reg_lambda=1.5; total time=   3.5s
[CV] END xgb__gamma=0.1, xgb__learning_rate=0.1, xgb__max_depth=3, xgb__n_estimators=5, xgb__reg_lambda=1.5; total time=   3.5s
[CV] END xgb__gamma=0.1, xgb__learning_rate=0.1, xgb__max_depth=3, xgb__n_estimators=5, xgb__reg_lambda=1.5; total time=   3.6s
[CV] END xgb__gamma=0.1, xgb__learning_rate=0.1, xgb__max_depth=3, xgb__n_estimators=10, xgb__reg_lambda=1.5; total time=   5.7s
[CV] END xgb__gamma=0.1, xgb__learning_rate=0.1, xgb__max_depth=3, xgb__n_estimators=10, xgb__reg_lambda=1.5; total time=   5.6s
[CV] END xgb__gamma=0.1, xgb__learning_rate=0.1, xgb__max_depth=3, xgb__n_estimators=10, xgb__reg_lamb

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('xgb',
                                              XGBClassifier(base_score=None,
                                                            booster=None,
                                                            callbacks=None,
                                                            colsample_bylevel=None,
                                                            colsample_bynode=None,
                                                            colsample_bytree=None,
                                                            early_stopping_rounds=None,
                                                            enable_categorical=False,
                                                            eval_metric='mlogloss',
                                                            gamma=None,
                                                            gpu_id=None,
                                                            grow_policy=None,
                                                            importance_type=None,
                                                            interaction_constraints=None,
                                                            lea...
                                                            monotone_constraints=None,
                                                            n_estimators=100,
                                                            n_jobs=None,
                                                            num_parallel_tree=None,
                                                            predictor=None,
                                                            random_state=None,
                                                            reg_alpha=None,
                                                            reg_lambda=None, ...))]),
                   n_iter=5, n_jobs=-1,
                   param_distributions={'xgb__gamma': [0.1],
                                        'xgb__learning_rate': [0.1],
                                        'xgb__max_depth': [3],
                                        'xgb__n_estimators': [5, 10],
                                        'xgb__reg_lambda': [1.5]},
                   random_state=0, scoring='accuracy', verbose=2)

In [112]:
# Évaluer le meilleur modèle sur les données de test
best_model = random_search.best_estimator_
print(f"Best Parameters: {random_search.best_params_}")
y_pred_train = best_model.predict(X_train_pca)
y_pred_test = best_model.predict(X_test_pca)
accuracy_train = accuracy_score(y_train, y_pred_train)
accuracy_test = accuracy_score(y_test, y_pred_test)
print(f"Accuracy Train = {accuracy_train}")
print(f"Accuracy Test = {accuracy_test}")

Best Parameters: {'xgb__reg_lambda': 1.5, 'xgb__n_estimators': 10, 'xgb__max_depth': 3, 'xgb__learning_rate': 0.1, 'xgb__gamma': 0.1}
Accuracy Train = 0.609
Accuracy Test = 0.062
